In [ ]:
pip install tensorflow

In [ ]:
import pickle
import numpy as np
import tensorflow
from keras.models import Sequential
from keras.layers import Dense

In [4]:
with open("/content/train_qa.txt",'rb') as fp:
  train_data = pickle.load(fp)

In [7]:
with open("/content/test_qa.txt",'rb') as fp:
  test_data = pickle.load(fp)

In [8]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [6]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [9]:
vocab=set()
all_data=train_data+test_data
for story,question,answer in all_data:
  vocab=vocab.union(set(story))
  vocab=vocab.union(set(question))
vocab.add('no')
vocab.add('yes')

In [10]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [12]:
vocab_len=len(vocab)+1
max_story_len=max([len(data[0]) for data in all_data])
max_question_len=max([len(data[1]) for data in all_data])

In [13]:
max_story_len

156

In [14]:
max_question_len

6

In [16]:
vocab_len

38

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
vocab_size=len(tokenizer.word_index)+1
print("Vocab_size :",vocab_size)

Vocab_size : 1


In [19]:
import keras

In [71]:
from keras.preprocessing.sequence import pad_sequences
text=['Mary moved to the bathroom.','John went to the hallway.']
tokenizer=Tokenizer(num_words=10)
tokenizer.fit_on_texts(text)
sequences=tokenizer.texts_to_sequences(text)
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)
vocab_size = len(tokenizer.word_index) + 2 # +1 for padding, +1 for '<unk>'
print(f"Corrected Vocab_size: {vocab_size}")
tokenizer.word_index

Corrected Vocab_size: 39


{'took': 1,
 'mary': 2,
 'back': 3,
 'kitchen': 4,
 'picked': 5,
 'football': 6,
 'moved': 7,
 'to': 8,
 'there': 9,
 'in': 10,
 'sandra': 11,
 'grabbed': 12,
 'journeyed': 13,
 'garden': 14,
 'apple': 15,
 'left': 16,
 'milk': 17,
 'put': 18,
 'travelled': 19,
 'office': 20,
 'up': 21,
 'bathroom': 22,
 'discarded': 23,
 'went': 24,
 'got': 25,
 'john': 26,
 'dropped': 27,
 'bedroom': 28,
 '?': 29,
 'the': 30,
 'hallway': 31,
 'daniel': 32,
 'yes': 33,
 '.': 34,
 'down': 35,
 'is': 36,
 'no': 37}

In [26]:
train_story_text=[]
train_question_text=[]
train_answers=[]
for story,question,answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
train_story_seq=tokenizer.texts_to_sequences(train_story_text)
len(train_story_text)

10000

In [27]:
len(train_story_seq)

10000

In [29]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
  X=[]
  Xq=[]
  Y=[]
  for story,query,answer in data:
    if '<unk>' not in word_index:
      word_index['<unk>']=len(word_index)+1

    x=[word_index.get(word.lower(),word_index['<unk>']) for word in story]
    xq=[word_index.get(word.lower(),word_index['<unk>']) for word in query]
    sq=np.zeros(len(word_index)+1)
    sq[word_index['yes']]=1
    y=np.zeros(len(word_index)+1)
    y[word_index[answer]]=1
    X.append(x)
    Xq.append(xq)
    Y.append(y)
  return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

input_train,queries_train,answers_train=vectorize_stories(train_data)
input_train.shape

(10000, 156)

In [30]:
queries_train.shape

(10000, 6)

In [31]:
answers_train.shape

(10000, 39)

In [32]:
input_test,queries_test,answers_test=vectorize_stories(test_data)
input_test

array([[ 0,  0,  0, ..., 30, 28, 34],
       [ 0,  0,  0, ..., 30, 14, 34],
       [ 0,  0,  0, ..., 30, 14, 34],
       ...,
       [ 0,  0,  0, ..., 30, 15, 34],
       [ 0,  0,  0, ..., 30, 14, 34],
       [ 0,  0,  0, ..., 15,  9, 34]], dtype=int32)

In [33]:
answers_test

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
       497.,   0.,   0.,   0., 503.,   0.])

In [35]:
tokenizer.word_index['yes']

33

In [36]:
tokenizer.word_index['no']

37

In [46]:
from keras.models import Sequential,Model
from keras.layers import Embedding
from keras.layers import Input,Activation,Dense,Permute,Dropout
from keras.layers import add,dot,concatenate
from keras.layers import LSTM

In [38]:
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))

In [40]:
input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

In [53]:
input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

In [54]:
question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [55]:
input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

In [56]:
match=dot([input_encoded_m,question_encoded],axes=(2,2))
match=Activation('softmax')(match)

In [57]:
response=add([match,input_encoded_c])
response=Permute((2,1))(response)

In [59]:
answer=Dropout(0.5)(concatenate([response,question_encoded]))
answer=LSTM(32)(answer)
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)

In [60]:
answer=Activation('softmax')(answer)
model=Model([input_sequence,question],answer)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [74]:
history=model.fit([input_train,queries_train],answers_train,batch_size=32,epochs=30,validation_data=([input_test,queries_test],answers_test))

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - accuracy: 0.5107 - loss: 0.6975 - val_accuracy: 0.5030 - val_loss: 0.6939
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 72ms/step - accuracy: 0.5015 - loss: 0.6994 - val_accuracy: 0.5030 - val_loss: 0.6935
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.4981 - loss: 0.6966 - val_accuracy: 0.4970 - val_loss: 0.7126
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - accuracy: 0.5013 - loss: 0.6968 - val_accuracy: 0.5030 - val_loss: 0.6934
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 73ms/step - accuracy: 0.4931 - loss: 0.6960 - val_accuracy: 0.4970 - val_loss: 0.6959
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 39s 67ms/step - accuracy: 0.5036 - loss: 0.6955 - val_accuracy: 0.5030 - val_loss: 0.6933
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 72ms/step - accuracy: 0.4968 - loss: 0.6953 - val_accuracy: 0.4970 - val_loss: 0.7001
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 73ms/step - accuracy: 0.4919 - loss: 0.6969 - 

In [72]:
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,concatenate
from tensorflow.keras.models import Model
max_len_input=input_train.shape[1]
max_len_query=queries_train.shape[1]
input1=Input(shape=(max_len_input,))
x1=Embedding(vocab_size,64)(input1)
x1=LSTM(32)(x1)

input2=Input(shape=(max_len_query,))
x2=Embedding(vocab_size,64)(input2)
x2=LSTM(32)(x2)

merged=concatenate([x1,x2])
output=Dense(answers_train.shape[1],activation='softmax')(merged)

model=Model(inputs=[input1,input2],outputs=output)

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 156)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, 156, 64)   │      2,496 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, 6, 64)     │      2,496 │ input_layer_13[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 32)        │     12,416 │ embedding_10[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 32)        │     12,416 │ embedding_11[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 64)        │          0 │ lstm_4[0][0],     │
│ (Concatenate)       │                   │            │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 39)        │      2,535 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,359 (126.40 KB)

 Trainable params: 32,359 (126.40 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
filename='chatbot_120_epochs.keras'
model.save(filename)

In [77]:
model.load_weights(filename)
pred_results=model.predict([input_test,queries_test])

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


In [84]:
story=' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [83]:
query=' '.join(word for word in test_data[0][1])
print(story)

Is John in the kitchen ?


In [80]:
print("True Test Answer from Data is:",test_data[0][2])

True Test Answer from Data is: no


In [82]:
val_max=np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
  if val==val_max:
    k=key

print("Predicted answer is: ",k)
print("Probability of certainty was: ",pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.5529173


In [85]:
my_story="John travelled the bathroom last saturday. Sandra dropped the football in the garden."
my_story

'John travelled the bathroom last saturday. Sandra dropped the football in the garden.'

In [86]:
my_question="Did John travelled the bathroom last friday?"
my_question.split()

['Did', 'John', 'travelled', 'the', 'bathroom', 'last', 'friday?']

In [87]:
my_data=[(my_story,my_question,'yes')]
my_story,my_ques,my_ans=vectorize_stories(my_data)
pred_results=model.predict(([my_story,my_ques]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


In [89]:
val_max=np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
  if val==val_max:
    k=key

print("Predicted answer is: ",k)
print("Probability of certainty was: ",pred_results[0][val_max])

Predicted answer is:  yes
Probability of certainty was:  0.58706164
